In [1]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
import urllib.request
import zipfile
from lxml import etree
import re
from nltk.tokenize import word_tokenize, sent_tokenize

In [3]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/GaoleMeng/RNN-and-FFNN-textClassification/master/ted_en-20160408.xml", filename="ted_en-20160408.xml")

('ted_en-20160408.xml', <http.client.HTTPMessage at 0x2167e8e5b20>)

In [4]:
targetXML=open('ted_en-20160408.xml', 'r', encoding='UTF8')
# 예제의 경우 윈도우 바탕화면에서 작업하여서 'C:\Users\USER\Desktop\ted_en-20160408.xml'이 해당 파일의 경로.  
target_text = etree.parse(targetXML)
parse_text = '\n'.join(target_text.xpath('//content/text()'))
# xml 파일로부터 <content>와 </content> 사이의 내용만 가져온다.

content_text = re.sub(r'\([^)]*\)', '', parse_text)
# 정규 표현식의 sub 모듈을 통해 content 중간에 등장하는 (Audio), (Laughter) 등의 배경음 부분을 제거.
# 해당 코드는 괄호로 구성된 내용을 제거.

sent_text = sent_tokenize(content_text)
# 입력 코퍼스에 대해서 NLTK를 이용하여 문장 토큰화를 수행.

normalized_text = []
for string in sent_text:
     tokens = re.sub(r"[^a-z0-9]+", " ", string.lower())
     normalized_text.append(tokens)
# 각 문장에 대해서 구두점을 제거하고, 대문자를 소문자로 변환.

result = []
result = [word_tokenize(sentence) for sentence in normalized_text]
# 각 문장에 대해서 NLTK를 이용하여 단어 토큰화를 수행.

In [5]:
print('총 샘플의 개수 : {}'.format(len(result)))

총 샘플의 개수 : 273424


# GloVe 모델 불러오기

In [6]:
# pip install glove_python
!pip install glove_python_binary


[notice] A new release of pip is available: 23.1.2 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## 훈련 데이터로부터 GloVe에서 사용할 동시 등장 행렬 생성
- vocab(corpus, 단어사전) 을 만들어주는 Corpus도 불러온다.

In [7]:
from glove import Corpus, Glove

corpus = Corpus() 
corpus.fit(result, window=5)

- no_components : embedding 차원 
- corpus를 matrix 형태로 입력
- epoch = 50 으로 학습
- 학습에 이용할 쓰레드의 개수 = 4

In [8]:
glove = Glove(no_components=100, learning_rate=0.05)
glove.fit(corpus.matrix, epochs=20, no_threads=4, verbose=True)

Performing 20 training epochs with 4 threads
Epoch 0
Epoch 1
Epoch 2
Epoch 3
Epoch 4
Epoch 5
Epoch 6
Epoch 7
Epoch 8
Epoch 9
Epoch 10
Epoch 11
Epoch 12
Epoch 13
Epoch 14
Epoch 15
Epoch 16
Epoch 17
Epoch 18
Epoch 19


In [9]:
glove.add_dictionary(corpus.dictionary)

In [10]:
model_result1=glove.most_similar("man")
print(model_result1)

[('woman', 0.9672063208808533), ('guy', 0.8860754067151129), ('girl', 0.8586488013414852), ('young', 0.8555609122021308)]


In [11]:
model_result2=glove.most_similar("boy")
print(model_result2)

[('girl', 0.9387513102930749), ('woman', 0.8462711083824784), ('man', 0.826285680291586), ('kid', 0.8215136008491446)]


In [12]:
model_result3=glove.most_similar("university")
print(model_result3)

[('harvard', 0.8860250672571577), ('mit', 0.8507060238355442), ('cambridge', 0.8421971422151926), ('stanford', 0.8303296663715781)]


In [13]:
model_result4=glove.most_similar("water")
print(model_result4)

[('fresh', 0.8402159512689902), ('clean', 0.8368317289585537), ('air', 0.8362398754877222), ('food', 0.8011240662848547)]


In [14]:
model_result5=glove.most_similar("physics")
print(model_result5)

[('chemistry', 0.8788511840749098), ('beauty', 0.8634394303533324), ('economics', 0.8633339320628183), ('biology', 0.8624156278004312)]
